# Hyperparameter Tuning
Find the most acceptable parameter

In [1]:
from optimization.acs import ACS_VRP,ACS_TSP
from optimization.bso import BSO_VRP,BSO_TSP
from optimization.hybrid_acs_bso import ACSBSO_VRP,ACSBSO_TSP

from optimization.koneksi import ConDB

import sql_connection
import json
import time
import threading
import random

## Maximum Iteration

In [2]:
def timeout_function():
    print("Script timed out.")
    # You can perform any cleanup or additional actions here
    # Exit the script if needed

# Set the timeout duration in seconds
timeout_duration = 120

# Create a thread that runs the timeout_function after the specified duration
timeout_thread = threading.Timer(timeout_duration, timeout_function)

In [3]:
query = """SELECT 
                p.post_id,
                p.post_type,
                CASE
                    WHEN pj.pj_jam_buka = pj.pj_jam_tutup THEN "tutup"
                    ELSE "buka"
                END AS is_operate
            FROM 
                posts p
            LEFT JOIN
                posts_jadwal pj
                ON p.post_id = pj.pj_id_tempat AND pj.pj_hari = "minggu"
            """

df_location = sql_connection.read_from_sql(query)

C:\Users\rahma\Documents\kuliah\thesis\rahmat_code\travel-itinerary-recommendation\sql_connection.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dataset = pd.read_sql(query, mydb)


In [5]:
random.seed(10)
tourid = df_location[(df_location['post_type']=="location")&
                     (df_location['is_operate'] != "tutup")]['post_id'].values.tolist()
idhotel = df_location[df_location['post_type']=="hotel"]['post_id'].values.tolist()
idhotel = idhotel[random.randint(0,len(idhotel))]
dwaktu,dtarif,drating = 1,1,1
travel_days = 3

In [6]:
db = ConDB()

hotel = db.HotelbyID(idhotel)
tur = db.WisatabyID(tourid)
timematrix = db.TimeMatrixbyID(hotel._id,tourid)

In [7]:
# Function to be executed by each thread
def worker_function(thread_id, iteration):
    print(f"Testing iteration {iteration}")
    
    start = time.time()
    
    timeout_thread = threading.Timer(120, timeout_function)  # Set a 2-minute (120 seconds) timeout
    timeout_thread.start()

    acs_model = ACS_VRP(max_iter=iteration, random_state=10)
    acs_model.set_model(tur, hotel, timematrix, travel_days=travel_days, degree_waktu=dwaktu, degree_tarif=dtarif, degree_rating=drating)

    sol, fitness = acs_model.construct_solution()
    print(f"Get fitness = {fitness}")

    timeout_thread.cancel()
    end = time.time()
    print(f"Running time = {end - start}")
    print("==============================================")

# Function to handle a timeout
def timeout_function():
    print("Timeout occurred for this iteration.")
    # You can perform any cleanup or additional actions here
    # Continue to the next iteration

# List of iterations
n = [100, 200, 300, 400, 500]

# Loop through the iterations
for i in n:
    thread = threading.Thread(target=worker_function, args=(i, i))
    thread.start()
    thread.join()  # Wait for the thread to finish

print("All iterations have finished.")

Testing iteration 100
Get fitness = 0.5668301419385561
Running time = 40.78083872795105
Testing iteration 200
Get fitness = 0.5693547470798093
Running time = 90.93568778038025
Testing iteration 300
Timeout occurred for this iteration.
Get fitness = 0.5702918308965187
Running time = 147.43390345573425
Testing iteration 400
Timeout occurred for this iteration.
Get fitness = 0.5702918308965187
Running time = 151.09607529640198
Testing iteration 500
Timeout occurred for this iteration.
Get fitness = 0.5702918308965187
Running time = 144.6768672466278
All iterations have finished.


In [60]:
def get_var(strs):
    t = str(strs)
    print(t)
    tourid = []
    t_ = json.loads(t)
    for t in t_['idWisata']:
        tourid.append(t['id'])
    idhotel = int(t_['idhotel'][0])
    dwaktu = float(t_['degree'][0])
    dtarif = float(t_['degree'][2])
    drating = float(t_['degree'][1])
    travel_days = int(t_['travel_days'])
    return tourid,idhotel,dwaktu,dtarif,drating,travel_days

choice = '{"idWisata":[{"id":"1"},{"id":"2"},{"id":"3"},{"id":"4"},{"id":"5"},{"id":"6"},{"id":"7"},{"id":"8"},{"id":"9"},{"id":"10"},{"id":"11"},{"id":"12"},{"id":"13"},{"id":"14"},{"id":"15"},{"id":"16"},{"id":"17"},{"id":"18"},{"id":"19"},{"id":"20"},{"id":"21"},{"id":"22"},{"id":"23"},{"id":"24"},{"id":"25"},{"id":"26"},{"id":"27"},{"id":"28"},{"id":"29"},{"id":"69"}],"idhotel":["169"],"algo_url":"ant-colony-optimization-aco-tsp","degree":[1,1,1],"travel_days":365}'
tourid,idhotel,dwaktu,dtarif,drating,travel_days = get_var(choice)

{"idWisata":[{"id":"1"},{"id":"2"},{"id":"3"},{"id":"4"},{"id":"5"},{"id":"6"},{"id":"7"},{"id":"8"},{"id":"9"},{"id":"10"},{"id":"11"},{"id":"12"},{"id":"13"},{"id":"14"},{"id":"15"},{"id":"16"},{"id":"17"},{"id":"18"},{"id":"19"},{"id":"20"},{"id":"21"},{"id":"22"},{"id":"23"},{"id":"24"},{"id":"25"},{"id":"26"},{"id":"27"},{"id":"28"},{"id":"29"},{"id":"69"}],"idhotel":["169"],"algo_url":"ant-colony-optimization-aco-tsp","degree":[1,1,1],"travel_days":365}


In [6]:
db = ConDB()
rute_perhari = []
waktuDatang = []

start = time.time()

hotel = db.HotelbyID(idhotel)
tur = db.WisatabyID(tourid)
timematrix = db.TimeMatrixbyID(hotel._id,tourid)

In [61]:
acs_model = ACS_VRP(max_iter=500,max_idem=50,random_state = 10)
acs_model.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)

sol,fitness = acs_model.construct_solution()
fitness

KeyboardInterrupt: 

In [23]:
bso_model = BSO_VRP(random_state = 100)
bso_model.set_model(tur,hotel,timematrix,init_solution=[],travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)

sol_node,sol,fitness = bso_model.construct_solution()
fitness

0.7035172143974963